# Example that calculates induced velocity distribution using only numpy vectorization

## Setup environment

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from src import Wing, FlightCondition, WingPool, Simulation, PostProcessing, load_folder

airfoils_data, _ = load_folder("sample_airfoils")
# print(airfoils_data)

wing = Wing(
    spans=[1, 0.5],
    chords=[1, 0.5, 0.4],
    offsets=[0, 0, 0],
    twist_angles=[0, 0, 0],
    dihedral_angles=[0, 0, 0],
    airfoils=["NACA4424", "NACA4424", "NACA4412"],
    surface_name="wing",
    N_panels=12,
    distribution_type="cosine",
    sweep_check=False,
)
wing.generate_mesh()  # Generate wing simulation elements
flight_condition = FlightCondition(
    V_inf=20,
    nu=1.5e-5,
    rho=1.225,
    angles_of_attack=[alpha for alpha in range(1)],
    h=0,
    ground_effect_check=False,
)
wing.setup_airfoil_data(flight_condition, airfoils_data)  # Assign airfoil data to wing

# Link wing that will be simulated with flight condition
wing_pool = WingPool(wing_list=[wing], flight_condition=flight_condition)

In [2]:
from src.utils.velocity import get_induced_velocity_distribution
import numpy as np
import numpy.linalg as npla

collocation_points = wing.collocation_points
vertice_points = wing.vertice_points
mac = np.array(wing.cp_macs)
v_inf_array = np.array(flight_condition.v_inf_list[0])

v_ij_distr = get_induced_velocity_distribution(collocation_points, wing.cp_macs, vertice_points, v_inf_array, wing.surface_name, False, 0)
print(v_ij_distr)

# ind_velocities_dict = wing_pool.calculate_induced_velocities(v_inf_array)
# for k, v in ind_velocities_dict.items():
#     print(f"{k}: {v}")

[[[ 0.00000000e+00  0.00000000e+00 -1.03006797e+01]
  [ 0.00000000e+00  0.00000000e+00  2.46643103e+00]
  [ 0.00000000e+00  0.00000000e+00  3.51202322e-01]
  [ 0.00000000e+00  0.00000000e+00  1.15598361e-01]
  [ 0.00000000e+00  0.00000000e+00  5.07086028e-02]
  [ 0.00000000e+00  0.00000000e+00  2.49794778e-02]
  [ 0.00000000e+00  0.00000000e+00  1.19482021e-02]
  [ 0.00000000e+00  0.00000000e+00  3.57525142e-03]
  [ 0.00000000e+00  0.00000000e+00  6.14485919e-03]
  [ 0.00000000e+00  0.00000000e+00  1.17668415e-02]
  [ 0.00000000e+00  0.00000000e+00  8.76346525e-03]
  [ 0.00000000e+00  0.00000000e+00  3.00755070e-03]]

 [[ 0.00000000e+00  0.00000000e+00  6.50158342e-01]
  [ 0.00000000e+00  0.00000000e+00 -2.81196168e+00]
  [ 0.00000000e+00  0.00000000e+00  9.92177518e-01]
  [ 0.00000000e+00  0.00000000e+00  1.75049614e-01]
  [ 0.00000000e+00  0.00000000e+00  6.47034145e-02]
  [ 0.00000000e+00  0.00000000e+00  2.96433280e-02]
  [ 0.00000000e+00  0.00000000e+00  1.37002807e-02]
  [ 0.0000

## Using only numpy vectorization

In [12]:
import numpy as np

def get_induced_velocity_np(collocation_points, vertice_points, mac, v_inf_array):
    mac = mac[:, np.newaxis, np.newaxis]
    ri1j = (
        collocation_points[:, np.newaxis, :] - vertice_points[np.newaxis, :-1, :]
    )
    ri2j = (
        collocation_points[:, np.newaxis, :] - vertice_points[np.newaxis, 1:, :]
    )
    ri1j_abs = npla.norm(ri1j, axis=2)[:, :, np.newaxis]
    ri2j_abs = npla.norm(ri2j, axis=2)[:, :, np.newaxis]
    r1_cross_prod = np.cross(v_inf_array, ri1j)
    r2_cross_prod = np.cross(v_inf_array, ri2j)
    r1_dot_prod = np.sum(v_inf_array * ri1j, axis=2)[:, :, np.newaxis]
    r2_dot_prod = np.sum(v_inf_array * ri2j, axis=2)[:, :, np.newaxis]
    r12_cross_prod = np.cross(ri1j, ri2j)
    r12_dot_prod = np.sum(ri1j * ri2j, axis=2)[:, :, np.newaxis]

    velocities = (
        mac / (4 * np.pi)
        * (
            r2_cross_prod / (ri2j_abs * (ri2j_abs - r2_dot_prod))
            - r1_cross_prod / (ri1j_abs * (ri1j_abs - r1_dot_prod))
        )
    )
    bound_vortex_den = ri1j_abs * ri2j_abs * (ri1j_abs * ri2j_abs + r12_dot_prod)

    threshold = 1e-10
    mask = np.isclose(bound_vortex_den, 0, atol=threshold)

    with np.errstate(divide='ignore', invalid='ignore'):
        bound_vortex = np.where(
            mask,
            np.zeros(mask.shape),
            (mac / (4 * np.pi) * ((ri1j_abs + ri2j_abs) * r12_cross_prod / bound_vortex_den)),
        )
    velocities += bound_vortex
    return velocities

# print(ri1j)
# print(r1_cross_prod[0][0])
# print(np.cross(v_inf_array, [-0.00837341, 0.0669873,  0.        ]))
# print(r1_dot_prod)
# print(np.dot(v_inf_array, [-0.00837341, 0.0669873,  0.        ]))
# print(r12_dot_prod)
# print(np.dot(ri1j[0][1], ri2j[0][1]))
# print(velocities)
# print(mask)

import time
mac = np.array(wing.cp_macs)
length = range(100_000)
t0 = time.perf_counter()
for i in length:
    velocities = get_induced_velocity_np(collocation_points, vertice_points, mac, v_inf_array)
t1 = time.perf_counter()
print(f"tempo numpy: {t1 - t0:.6f}s")

# for i in length:
#     v_ij_distr = get_induced_velocity_distribution(
#         collocation_points,
#         wing.cp_macs,
#         vertice_points,
#         v_inf_array,
#         wing.surface_name,
#         False,
#         0,
#     )
# t2 = time.perf_counter()


# print(f"tempo normal: {t2 - t1:.6f}s")
# print(velocities)
# print(bound_vortex)

tempo numpy: 49.772595s


In [4]:
print(velocities-v_ij_distr)

[[[ 0.          0.          0.        ]
  [ 0.          0.         -0.06509815]
  [ 0.          0.         -0.01312657]
  [ 0.          0.         -0.01528892]]

 [[ 0.          0.          0.02536564]
  [ 0.          0.          0.        ]
  [ 0.          0.         -0.03437116]
  [ 0.          0.         -0.02485602]]

 [[ 0.          0.          0.01022959]
  [ 0.          0.          0.0535712 ]
  [ 0.          0.          0.        ]
  [ 0.          0.         -0.10962473]]

 [[ 0.          0.          0.00716598]
  [ 0.          0.          0.02360857]
  [ 0.          0.          0.02176784]
  [ 0.          0.          0.        ]]]


In [5]:
print(f'ri1j_abs: {ri1j_abs}')
print(f'ri2j_abs: {ri2j_abs}')
print(f'r1_cross_prod: {r1_cross_prod}')
print(f'r2_cross_prod: {r2_cross_prod}')
print(f'r1_dot_prod: {r1_dot_prod}')
print(f'r2_dot_prod: {r2_dot_prod}')
print(f'r12_cross_prod: {r12_cross_prod}')
print(f'r12_dot_prod: {r12_dot_prod}')

ri1j_abs: [[[0.06750861]
  [0.18443695]
  [0.68832806]
  [0.94027361]]

 [[0.50389111]
  [0.25194555]
  [0.25194555]
  [0.50389111]]

 [[0.94027361]
  [0.68832806]
  [0.18443695]
  [0.06750861]]

 [[1.25753976]
  [1.00562869]
  [0.50191041]
  [0.2503123 ]]]
ri2j_abs: [[[0.18443695]
  [0.68832806]
  [0.94027361]
  [1.43999427]]

 [[0.25194555]
  [0.25194555]
  [0.50389111]
  [1.00382083]]

 [[0.68832806]
  [0.18443695]
  [0.06750861]
  [0.56796864]]

 [[1.00562869]
  [0.50191041]
  [0.2503123 ]
  [0.2503123 ]]]
r1_cross_prod: [[[ 0.        -0.         0.0669873]
  [ 0.         0.        -0.1830127]
  [ 0.         0.        -0.6830127]
  [ 0.         0.        -0.9330127]]

 [[ 0.        -0.         0.5      ]
  [ 0.        -0.         0.25     ]
  [ 0.         0.        -0.25     ]
  [ 0.         0.        -0.5      ]]

 [[ 0.        -0.         0.9330127]
  [ 0.        -0.         0.6830127]
  [ 0.        -0.         0.1830127]
  [ 0.         0.        -0.0669873]]

 [[ 0.        -0.  

In [21]:
mac = np.array(wing.cp_macs)
mac = mac[:, np.newaxis, np.newaxis]
mac
# mac / (4 * np.pi)

array([[[0.93888889]],

       [[0.75694444]],

       [[0.56481481]],

       [[0.45185185]]])